# LangChain RAG Project with Pride and Prejudice book

## Load ENV variables

In [1]:
import langchain
print(langchain.__version__)

0.3.0


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import torch

In [4]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

/Users/elviramagallanes/envs/langchain_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: Module langchain_community.document_loaders not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')

## Documents loaders

In [ ]:
loader = TextLoader('Data/pride_and_prejudice.txt', encoding='utf-8')
documents = loader.load()

### Text splitting

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
docs = text_splitter.split_documents(documents)

### Embeddings

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

embeddings = HuggingFaceEmbeddings(
    model_name="nvidia/NV-Embed-v2",
    model_kwargs={'device': device},
)

### Create our vector storage

In [ ]:
vector_store = FAISS.from_documents(docs, embeddings)

In [ ]:
vector_store.save_local("faiss_index")

In [ ]:
vector_store = FAISS.load_local("faiss_index", embeddings)

In [ ]:
# Retriever
retriever = vector_store.as_retriever()

### LLM

In [ ]:
llm = OpenAI(openai_api_key=openai_api_key)